1d/2d/3d Convolution  

卷积运算：卷积核在输入信号（图像）上滑动，相应位置进行乘加  
卷积核：又称为滤波器，过滤器，可认为是某种模式，某种特征  
卷积过程类似于用一个模版去图像上寻找与它相似的区域，与卷积核模式越相似，激活值越高，从而实现特征提取  

nn.Conv2d  
功能：对多个二维信号进行二维卷积  
* in_channels:输入通道数  
* out_channels:输出通道数，等价于卷积核个数  
* kernel_size:卷积核尺寸  
* stride:步长  
* padding:填充个数  
* dilation:空洞卷积大小  
* groups:分组卷积设置  
* bias:偏置  
尺寸计算：  
简化版：输出尺寸 = （输入尺寸-卷积核大小）/步长+1  
完整版：输出尺寸 = （输入尺寸+2 * padding-dilation *（卷积核大小-1）-1）/步长+1  


In [13]:
#-*-coding:utf-8-*-
import os
import torch.nn as nn
from PIL import Image
from torchvision import transforms
from matplotlib import pyplot as plt
import inspect
import os.path
from pytorch_code.lesson3.tools.transform_invert import set_seed,transform_invert
set_seed(3)

path_img = os.path.join(os.getcwd(), 'pytorch_code', 'lesson3', "lena.png")
img = Image.open(path_img).convert('RGB')

img_transfrom = transforms.Compose([transforms.ToTensor()])
img_tensor = img_transfrom(img)
img_tensor.unsqueeze_(dim=0)

tensor([[[[0.8824, 0.8824, 0.8824,  ..., 0.8941, 0.8549, 0.7961],
          [0.8784, 0.8824, 0.8784,  ..., 0.9059, 0.8588, 0.7922],
          [0.8824, 0.8784, 0.8784,  ..., 0.9137, 0.8667, 0.7765],
          ...,
          [0.3216, 0.3098, 0.3686,  ..., 0.6863, 0.6824, 0.6824],
          [0.3216, 0.3137, 0.3843,  ..., 0.7059, 0.7137, 0.7059],
          [0.3255, 0.3176, 0.3882,  ..., 0.7020, 0.7216, 0.7216]],

         [[0.5412, 0.5333, 0.5333,  ..., 0.5843, 0.5176, 0.3922],
          [0.5333, 0.5333, 0.5333,  ..., 0.5882, 0.5216, 0.3922],
          [0.5373, 0.5373, 0.5373,  ..., 0.5765, 0.5098, 0.3804],
          ...,
          [0.0863, 0.0706, 0.1176,  ..., 0.2706, 0.2588, 0.2588],
          [0.0863, 0.0745, 0.1333,  ..., 0.2745, 0.2824, 0.2863],
          [0.0902, 0.0784, 0.1373,  ..., 0.2667, 0.2941, 0.2941]],

         [[0.4745, 0.5020, 0.5176,  ..., 0.4627, 0.4196, 0.3333],
          [0.4784, 0.5020, 0.5176,  ..., 0.4745, 0.4314, 0.3451],
          [0.4902, 0.5020, 0.5098,  ..., 0

In [10]:
conv_layer = nn.Conv2d(3, 1, 3)
nn.init.xavier_normal_(conv_layer.weight.data)
img_conv = conv_layer(img_tensor)

In [11]:
img_tensor.shape, img_conv.shape, img_conv[0, 0:1, ...].shape

(torch.Size([1, 3, 512, 512]),
 torch.Size([1, 1, 510, 510]),
 torch.Size([1, 510, 510]))

In [12]:
img_conv = transform_invert(img_conv[0, 0:1, ...], img_transfrom)
img_raw = transform_invert(img_tensor.squeeze(), img_transfrom)
plt.subplot(122).imshow(img_conv, cmap='gray')
plt.subplot(121).imshow(img_raw)
plt.show()

In [5]:
conv_layer = nn.ConvTranspose2d(3, 1, 3, stride=2)
nn.init.xavier_normal_(conv_layer.weight.data)
img_conv = conv_layer(img_tensor)

In [6]:
img_tensor.shape, img_conv.shape, img_conv[0, 0:1, ...].shape

(torch.Size([1, 3, 512, 512]),
 torch.Size([1, 1, 1025, 1025]),
 torch.Size([1, 1025, 1025]))

In [7]:
img_conv = transform_invert(img_conv[0, 0:1, ...], img_transfrom)
img_raw = transform_invert(img_tensor.squeeze(), img_transfrom)
plt.subplot(122).imshow(img_conv, cmap='gray')
plt.subplot(121).imshow(img_raw)
plt.show()

转置卷积又称为反卷积和部分跨越卷积，用于对图像进行上采样  
nn.ConvTranspose2d  
功能：转置卷积实现上采样  
主要参数：  
* in_channels:输入通道数  
* out_channels:输出通道数，等价于卷积核个数  
* kernel_size:卷积核尺寸  
* stride:步长  
* padding:填充个数  
* dilation:空洞卷积大小  
* groups:分组卷积设置  
* bias:偏置  
尺寸计算：  
简化版：输出尺寸 =（输入尺寸-1）* 步长 + 卷积核大小  
完整版：输出尺寸 = (输入尺寸-1) * 步长 - 2 * padding + dilation * (卷积核大小 - 1) + output_padding + 1  
转置卷积有棋盘效应